### **HuggingFace Login**

In [43]:
from huggingface_hub import login
login()

### **Import Libairies**

In [56]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from datasets import Dataset



### **Read Datasets**

In [45]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
'''

df_train = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv", index_col=0)
df_test = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv", index_col=0)


### **Short Analysis**

In [46]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


### **Preprocess Datasets**

##### _Merge columns (full)_

In [47]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
sinking,41,9
damage,41,9


In [48]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [49]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


### **Model Assessment**

In [52]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Value': [f1, precision, recall]
    }, index=['F1 Score', 'Precision', 'Recall'])
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

### **DISTIL BERT Tokenizer**

##### _DISTIL BERT Tokenizer + split validation_

In [ ]:
distill_bert_tokenizer_full = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

distill_bert_encodings_full = distill_bert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
distill_bert_labels_full = torch.tensor(list(df_train_full['target']))

distill_bert_input_ids_train_full, distill_bert_input_ids_valid_full, \
distill_bert_attention_mask_train_full, distill_bert_attention_mask_valid_full, distill_bert_y_train_full, distill_bert_y_valid_full = train_test_split(
    distill_bert_encodings_full['input_ids'], 
    distill_bert_encodings_full['attention_mask'], 
    distill_bert_labels_full, 
    test_size=0.15, 
    stratify=distill_bert_labels_full, 
    random_state=0
)

distill_bert_train_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_train_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_train_full)
}

distill_bert_valid_encodings_full = {
    'input_ids': torch.tensor(distill_bert_input_ids_valid_full),
    'attention_mask': torch.tensor(distill_bert_attention_mask_valid_full)
}

distill_bert_train_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_train_encodings_full['input_ids'],
    "attention_mask": distill_bert_train_encodings_full['attention_mask'],
    "labels": distill_bert_y_train_full
})

distill_bert_valid_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_valid_encodings_full['input_ids'],
    "attention_mask": distill_bert_valid_encodings_full['attention_mask'],
    "labels": distill_bert_y_valid_full
})

distill_bert_test_encodings_full = distill_bert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)

distill_bert_test_encodings_full = {
    key: torch.tensor(val) for key, val in distill_bert_test_encodings_full.items()
    }

distill_bert_test_dataset_full = Dataset.from_dict({
    "input_ids": distill_bert_test_encodings_full['input_ids'],
    "attention_mask": distill_bert_test_encodings_full['attention_mask']
})

##### _Save HuggingFace DISTIL BERT Tokenizer_

In [ ]:
distill_bert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_tokenizer")

### **DISTIL BERT Training from scratch**

##### _DISTIL BERT Transformers + local save_

In [ ]:
distill_bert_model_full = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./distil_bert_results",
    num_train_epochs=6,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./distil_bert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    report_to="none",
    no_cuda=True 
)

distill_bert_trainer_full = Trainer(
    model=distill_bert_model_full,
    args=training_args,
    train_dataset=distill_bert_train_dataset_full,
    eval_dataset=distill_bert_valid_dataset_full
)

distill_bert_trainer_full.train()

distill_bert_trainer_full.save_model("distil_bert_model") # Local

##### _Save HugggingFace trained DISTIL BERT Transformers_ 

In [ ]:
distill_bert_trainer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_model")
# distill_bert_trainer_full.model.save_pretrained("yanncauchepin/kaggle_disastertweets_distill_bert_trainer")

### **DISTIL BERT Loading from pretrained**

In [ ]:
distill_bert_model_full = DistilBertForSequenceClassification.from_pretrained("distil_bert_model")

distil_bert_trainer_full = Trainer(
    model=distill_bert_model_full
)

### **DISTIL BERT Assessment**

In [ ]:
distill_bert_predictions_full = distill_bert_trainer_full.predict(distill_bert_valid_dataset_full)
distill_bert_logits_full = distill_bert_predictions_full.predictions
distill_bert_y_pred_full = np.argmax(distill_bert_logits_full, axis=1)

distill_bert_trainer_full_assessement = evaluate_classifier(distill_bert_y_valid_full.numpy(), distill_bert_y_pred_full)
display(distill_bert_trainer_full_assessement[0])
display(distill_bert_trainer_full_assessement[1])

### **DISTIL BERT Test Predictions**

##### _DISTIL BERT Test Predictions_

In [ ]:
distill_bert_test_predictions_full = distill_bert_trainer_full.predict(distill_bert_test_dataset_full)
distill_bert_test_logits_full = distill_bert_test_predictions_full.predictions
distill_bert_test_y_pred_full = np.argmax(distill_bert_test_logits_full, axis=1)

distill_bert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': distill_bert_test_y_pred_full.flatten()
})

##### _Save HuggingFace DISTIL BERT Test Predictions_

In [ ]:
hf_distill_bert_test_submission_full = Dataset.from_pandas(distill_bert_test_submission_full)
hf_distill_bert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_distill_bert_submission_df")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

...

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.7988894581794739, 'eval_runtime': 47.8857, 'eval_samples_per_second': 23.848, 'eval_steps_per_second': 5.973, 'epoch': 3.0}
{'train_runtime': 5066.6684, 'train_samples_per_second': 3.832, 'train_steps_per_second': 0.958, 'train_loss': 0.41929942678750715, 'epoch': 3.0}


  0%|          | 0/286 [00:00<?, ?it/s]

  0%|          | 0/816 [00:00<?, ?it/s]

AttributeError: 'Trainer' object has no attribute 'save_pretrained'